<a href="https://colab.research.google.com/github/Pinedah/ESCOM_development-of-applications-for-data-analysis/blob/main/Untitled6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving MBA.csv to MBA.csv


In [ ]:
import pandas as pd  # Para trabajar con la mayoría de los formatos
import json          # Para exportar a JSON
import xml.etree.ElementTree as ET  # Para exportar a XML
import h5py          # Para exportar a HDF5
import openpyxl      #Para archivos excel

# Función para leer los archivos y convertirlos después a otro formato.
def conversor(archivo, tipo_archivo, formato_conversión, archivo_convertido):
    if tipo_archivo == formato_conversión:  # En caso de que el formato a convertir sea el mismo del archivo original.
        print(f"Tu archivo {archivo} ya está en formato {formato_conversión}.")
        return

    # Esta parte se encarga de leer el archivo según su tipo:
    try:
        # Leer un archivo csv
        if tipo_archivo == "csv":
            df = pd.read_csv(archivo)
        # Leer un archivo tsv
        elif tipo_archivo == "tsv":
            df = pd.read_csv(archivo, sep="\t")
        # Leer un archivo excel
        elif tipo_archivo == "excel":
            df = pd.read_excel(archivo)
        # Leer un archivo json
        elif tipo_archivo == "json":
            with open(archivo, 'r') as f:
                df = pd.DataFrame(json.load(f))
        # Leer un archivo xml
        elif tipo_archivo == "xml":
            tree = ET.parse(archivo)
            root = tree.getroot()
            data = []
            for child in root:
                data.append({element.tag: element.text for element in child})
            df = pd.DataFrame(data)  # Convertir la estructura de XML a DataFrame
        # Leer un archivo hdf5
        elif tipo_archivo == "hdf5":
            df = {}
            with h5py.File(archivo, 'r') as f:
                for key in f.keys():
                    df[key] = f[key][:]  # Guardar cada dataset en un diccionario
            df = pd.DataFrame(df)  # Convertir el diccionario a DataFrame
        else:
            raise ValueError(f"No se puede convertir tu archivo desde {tipo_archivo} al formato {formato_conversión}")

    except FileNotFoundError:
        print(f"El archivo '{archivo}' no se encuentra en el directorio actual.")
        return

    # Esta parte de la función se encarga de convertir el archivo al formato deseado:
    try:
       #Comvertir a csv
        if formato_conversión == "csv":
            df.to_csv(archivo_convertido, index=False)
        #Convertir a tsv
        elif formato_conversión == "tsv":
            df.to_csv(archivo_convertido, sep="\t", index=False)
        #Convertir a excel
        elif formato_conversión == "excel":
            df.to_excel(archivo_convertido, index=False)
        #Convertir a json
        elif formato_conversión == "json":
          # Verifica si alguna columna contiene objetos de tipo bytes y conviértelos a string
            for col in df.columns:
              if df[col].dtype == 'object':
                 df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
            with open(archivo_convertido, 'w') as f:
                json.dump(df.to_dict(orient="records"), f)
        #Convertir a xml
        elif formato_conversión == "xml":
            root = ET.Element("root")
            for _, row in df.iterrows():
                item = ET.SubElement(root, "item")
                for col, val in row.items():
                    child = ET.SubElement(item, col)
                    child.text = str(val)
            tree = ET.ElementTree(root)
            tree.write(archivo_convertido)
        #Convertir a hdf5
        elif formato_conversión == "hdf5":
           # Convertir columnas de tipo object a string (Esto por el caso de hdf5 que no acepta las columnas object)
            for col in df.select_dtypes(include=['object']).columns:
                 df[col] = df[col].astype(str)
            with h5py.File(archivo_convertido, 'w') as f:
                for col in df.columns:
                    f.create_dataset(col, data=df[col].values)
        else:
            raise ValueError(f"El formato {formato_conversión} no es válido.")
        print(f"Archivo convertido exitosamente a {formato_conversión} como '{archivo_convertido}'.")
    except Exception as e:
        print(f"Error: {str(e)}")

# Función principal donde se piden los datos al usuario
def main():
    archivo = input("Selecciona el nombre del archivo que deseas manipular. Ejemplo: hola.csv\n")
    tipo_archivo = input("¿Cuál es el formato de tu archivo? (CSV, TSV, Excel, JSON, XML, HDF5)\n").lower()

    formatos_validos = ["csv", "tsv", "excel", "json", "xml", "hdf5"]
    formatos_punto = ["CSV: .csv", "TSV: .tsv", "Excel: .xlsx", "JSON: .json", "XML: .xml", "HDF5: .h5"]

    if tipo_archivo in formatos_validos:
        formato_conversión = input(f"¿A qué formato deseas hacer la conversión? {formatos_validos}\n").lower()
        if formato_conversión not in formatos_validos:
            print(f"Formato no válido. Debe ser uno de los siguientes: {formatos_validos}")
            return

        archivo_convertido = input(f"¿Cómo quieres que se llame tu archivo convertido? Ejemplo: hola.tsv\n Las extensiones son: {formatos_punto}")
        conversor(archivo, tipo_archivo, formato_conversión, archivo_convertido)
    else:
        print(f"Formato no reconocido. Los formatos válidos son: {formatos_validos}")

if __name__ == '__main__':
    main()
